In [ ]:
#@title Check GPU
%tensorflow_version 2.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [ ]:
#@title Version Info
print('tf version: ', tf.__version__)
print('tf.keras version:', tf.keras.__version__)


tf version:  2.5.0
tf.keras version: 2.5.0


#References:
[tf.keras.layers.LSTM official website](https://www.tensorflow.org/api_docs/python/tf/keras/layers/LSTM)

[The base example](https://machinelearningmastery.com/encoder-decoder-attention-sequence-to-sequence-prediction-keras/)

[My Presentation in PPT format]()


In [ ]:
#@title Import Libraries
from random import randint
from numpy import array
from numpy import argmax

from tensorflow.keras import models
from numpy import array_equal
import numpy as np
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras import Input
from tensorflow.keras.layers import TimeDistributed
from tensorflow.keras.layers import RepeatVector

In [ ]:
#@title Generate one_hot_encoded Input & Output Sequences


# generate a sequence of random integers
def generate_sequence(length, n_unique):
	return [randint(0, n_unique-1) for _ in range(length)]

# one hot encode sequence
def one_hot_encode(sequence, n_unique):
	encoding = list()
	for value in sequence:
		vector = [0 for _ in range(n_unique)]
		vector[value] = 1
		encoding.append(vector)
	return array(encoding)

# decode a one hot encoded string
def one_hot_decode(encoded_seq):
	return [argmax(vector) for vector in encoded_seq]

# prepare data for the LSTM
def get_pair(n_in, n_out, n_unique, verbose= False):
	# generate random sequence
	sequence_in = generate_sequence(n_in, n_unique)
	sequence_out = sequence_in[:n_out] + [0 for _ in range(n_in-n_out)]
	# one hot encode
	X = one_hot_encode(sequence_in, n_unique)
	y = one_hot_encode(sequence_out, n_unique)
	# reshape as 3D
	X = X.reshape((1, X.shape[0], X.shape[1]))
	y = y.reshape((1, y.shape[0], y.shape[1]))

	if(verbose):
		print('Generated sequences as follows')
		print('X.shape: ', X.shape,'y.shape: ', y.shape)
		print('\nSample X and y')
		print('\nIn raw format:')
		print('X=%s, y=%s' % (one_hot_decode(X[0]), one_hot_decode(y[0])))
		print('\nIn one_hot_encoded format:')
		print('X=%s' % (X[0]))
	return X,y

In [ ]:
#@title Configure problem

n_timesteps_in = 4  #@param {type:"integer"}
#each input sample has 4 values

n_features = 10   #@param {type:"integer"}
#each value is one_hot_encoded with 10 0/1
n_timesteps_out = 2  #@param {type:"integer"}
#each output sample has 2 values padded with 0

# generate random sequence
X,y = get_pair(n_timesteps_in, n_timesteps_out, n_features, verbose=True)

Generated sequences as follows
X.shape:  (1, 4, 10) y.shape:  (1, 4, 10)

Sample X and y

In raw format:
X=[2, 9, 4, 8], y=[2, 9, 0, 0]

In one_hot_encoded format:
X=[[0 0 1 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 1]
 [0 0 0 0 1 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 1 0]]


In [ ]:
# define model
numberOfLSTMcells= 100

input =Input(shape=(n_timesteps_in, n_features))
output= LSTM(numberOfLSTMcells, return_state=True) (input)
model1 = Model(inputs=input, outputs=output)
model1.compile(loss='categorical_crossentropy', optimizer='adam', 
              metrics=['accuracy'])

model1.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 4, 10)]           0         
_________________________________________________________________
lstm (LSTM)                  [(None, 100), (None, 100) 44400     
Total params: 44,400
Trainable params: 44,400
Non-trainable params: 0
_________________________________________________________________


In [ ]:
output


[<KerasTensor: shape=(None, 100) dtype=float32 (created by layer 'lstm')>,
 <KerasTensor: shape=(None, 100) dtype=float32 (created by layer 'lstm')>,
 <KerasTensor: shape=(None, 100) dtype=float32 (created by layer 'lstm')>]

In [ ]:
#lstm, state_h, state_c

In [ ]:
# define model
model = Sequential()
model.add(Input(shape=(n_timesteps_in, n_features)))
model.add(LSTM(150))
model.add(RepeatVector(n_timesteps_in))
model.add(LSTM(150, return_sequences=True))
model.add(TimeDistributed(Dense(n_features, activation='softmax')))
model.compile(loss='categorical_crossentropy', optimizer='adam', 
              metrics=['accuracy'])
model.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 150)               96600     
_________________________________________________________________
repeat_vector (RepeatVector) (None, 4, 150)            0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 4, 150)            180600    
_________________________________________________________________
time_distributed (TimeDistri (None, 4, 10)             1510      
Total params: 278,710
Trainable params: 278,710
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# train LSTM
for epoch in range(500):
	# generate new random sequence
	X,y = get_pair(n_timesteps_in, n_timesteps_out, n_features)
	# fit model for one epoch on this sequence
	model.fit(X, y, epochs=1, verbose=2)
	print(epoch,' finished...')

1/1 - 0s - loss: 2.3064 - accuracy: 0.0000e+00
0  finished...
1/1 - 0s - loss: 2.2981 - accuracy: 0.5000
1  finished...
1/1 - 0s - loss: 2.2849 - accuracy: 0.5000
2  finished...
1/1 - 0s - loss: 2.2575 - accuracy: 0.5000
3  finished...
1/1 - 0s - loss: 2.2586 - accuracy: 0.5000
4  finished...
1/1 - 0s - loss: 2.2631 - accuracy: 0.5000
5  finished...
1/1 - 0s - loss: 2.2313 - accuracy: 0.5000
6  finished...
1/1 - 0s - loss: 2.2234 - accuracy: 0.5000
7  finished...
1/1 - 0s - loss: 2.2140 - accuracy: 0.5000
8  finished...
1/1 - 0s - loss: 2.1577 - accuracy: 0.5000
9  finished...
1/1 - 0s - loss: 2.1474 - accuracy: 0.5000
10  finished...
1/1 - 0s - loss: 2.1132 - accuracy: 0.7500
11  finished...
1/1 - 0s - loss: 2.0560 - accuracy: 0.5000
12  finished...
1/1 - 0s - loss: 2.0303 - accuracy: 0.5000
13  finished...
1/1 - 0s - loss: 1.9488 - accuracy: 0.7500
14  finished...
1/1 - 0s - loss: 1.9460 - accuracy: 0.5000
15  finished...
1/1 - 0s - loss: 1.8788 - accuracy: 0.5000
16  finished...
1/1

In [ ]:
# evaluate LSTM
total, correct = 100, 0
for _ in range(total):
	X,y = get_pair(n_timesteps_in, n_timesteps_out, n_features)
	yhat = model.predict(X, verbose=0)
	if array_equal(one_hot_decode(y[0]), one_hot_decode(yhat[0])):
		correct += 1
print('Accuracy: %.2f%%' % (float(correct)/float(total)*100.0))

Accuracy: 13.00%


In [ ]:
# spot check some examples
for _ in range(10):
	X,y = get_pair(n_timesteps_in, n_timesteps_out, n_features)
	yhat = model.predict(X, verbose=0)
	print('Input',one_hot_decode(X[0]),
	      'Expected:', one_hot_decode(y[0]), 
				   'Predicted', one_hot_decode(yhat[0]))

Input [8, 0, 3, 4] Expected: [8, 0, 0, 0] Predicted [8, 0, 0, 0]
Input [7, 0, 5, 5] Expected: [7, 0, 0, 0] Predicted [7, 7, 0, 0]
Input [0, 1, 7, 2] Expected: [0, 1, 0, 0] Predicted [1, 1, 0, 0]
Input [8, 9, 4, 5] Expected: [8, 9, 0, 0] Predicted [8, 4, 0, 0]
Input [5, 1, 8, 9] Expected: [5, 1, 0, 0] Predicted [1, 1, 0, 0]
Input [5, 1, 6, 3] Expected: [5, 1, 0, 0] Predicted [1, 1, 0, 0]
Input [2, 0, 2, 5] Expected: [2, 0, 0, 0] Predicted [2, 2, 0, 0]
Input [4, 5, 1, 5] Expected: [4, 5, 0, 0] Predicted [1, 4, 0, 0]
Input [4, 5, 7, 1] Expected: [4, 5, 0, 0] Predicted [1, 1, 0, 0]
Input [0, 7, 5, 8] Expected: [0, 7, 0, 0] Predicted [7, 0, 0, 0]


In [ ]:
x= tf.constant([[3,4,5]])
print(x)
tf.keras.layers.RepeatVector(3)(x)

tf.Tensor([[3 4 5]], shape=(1, 3), dtype=int32)


<tf.Tensor: shape=(1, 3, 3), dtype=int32, numpy=
array([[[3, 4, 5],
        [3, 4, 5],
        [3, 4, 5]]], dtype=int32)>

In [ ]:
# define model
model3 = Sequential()
model3.add(Input(shape=(n_timesteps_in, n_features)))
model3.add(LSTM(150, return_sequences=True))
model3.add(LSTM(150, return_sequences=True))
model3.add(TimeDistributed(Dense(n_features, activation='softmax')))
model3.compile(loss='categorical_crossentropy', optimizer='adam', 
               metrics=['accuracy'])
model3.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_3 (LSTM)                (None, 4, 150)            96600     
_________________________________________________________________
lstm_4 (LSTM)                (None, 4, 150)            180600    
_________________________________________________________________
time_distributed_1 (TimeDist (None, 4, 10)             1510      
Total params: 278,710
Trainable params: 278,710
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# train LSTM
for epoch in range(1500):
	# generate new random sequence
	X,y = get_pair(n_timesteps_in, n_timesteps_out, n_features)
	# fit model for one epoch on this sequence
	model3.fit(X, y, epochs=1, verbose=2)
	print(epoch,' finished...')

1/1 - 0s - loss: 2.3029 - accuracy: 0.2500
0  finished...
1/1 - 0s - loss: 2.2951 - accuracy: 0.5000
1  finished...
1/1 - 0s - loss: 2.2746 - accuracy: 0.5000
2  finished...
1/1 - 0s - loss: 2.2794 - accuracy: 0.5000
3  finished...
1/1 - 0s - loss: 2.2629 - accuracy: 0.7500
4  finished...
1/1 - 0s - loss: 2.2680 - accuracy: 0.5000
5  finished...
1/1 - 0s - loss: 2.2279 - accuracy: 0.5000
6  finished...
1/1 - 0s - loss: 2.2236 - accuracy: 0.5000
7  finished...
1/1 - 0s - loss: 2.2147 - accuracy: 0.5000
8  finished...
1/1 - 0s - loss: 2.2070 - accuracy: 0.5000
9  finished...
1/1 - 0s - loss: 2.1844 - accuracy: 0.5000
10  finished...
1/1 - 0s - loss: 2.1312 - accuracy: 0.5000
11  finished...
1/1 - 0s - loss: 2.1465 - accuracy: 0.5000
12  finished...
1/1 - 0s - loss: 2.1130 - accuracy: 0.5000
13  finished...
1/1 - 0s - loss: 2.0824 - accuracy: 0.5000
14  finished...
1/1 - 0s - loss: 2.0124 - accuracy: 0.7500
15  finished...
1/1 - 0s - loss: 2.0014 - accuracy: 0.5000
16  finished...
1/1 - 0

In [ ]:
# evaluate LSTM
total, correct = 100, 0
for _ in range(total):
	X,y = get_pair(n_timesteps_in, n_timesteps_out, n_features)
	yhat = model3.predict(X, verbose=0)
	if array_equal(one_hot_decode(y[0]), one_hot_decode(yhat[0])):
		correct += 1
print('Accuracy: %.2f%%' % (float(correct)/float(total)*100.0))

Accuracy: 100.00%


In [ ]:
# spot check some examples
for _ in range(10):
	X,y = get_pair(n_timesteps_in, n_timesteps_out, n_features)
	yhat = model3.predict(X, verbose=0)
	print('Input',one_hot_decode(X[0]),
	      'Expected:', one_hot_decode(y[0]), 
				   'Predicted', one_hot_decode(yhat[0]))

Input [1, 6, 8, 7] Expected: [1, 6, 0, 0] Predicted [1, 6, 0, 0]
Input [5, 6, 7, 6] Expected: [5, 6, 0, 0] Predicted [5, 6, 0, 0]
Input [8, 6, 0, 1] Expected: [8, 6, 0, 0] Predicted [8, 6, 0, 0]
Input [6, 4, 3, 2] Expected: [6, 4, 0, 0] Predicted [6, 4, 0, 0]
Input [8, 7, 8, 3] Expected: [8, 7, 0, 0] Predicted [8, 7, 0, 0]
Input [7, 5, 5, 9] Expected: [7, 5, 0, 0] Predicted [7, 5, 0, 0]
Input [2, 0, 1, 9] Expected: [2, 0, 0, 0] Predicted [2, 0, 0, 0]
Input [4, 1, 0, 7] Expected: [4, 1, 0, 0] Predicted [4, 1, 0, 0]
Input [5, 7, 9, 2] Expected: [5, 7, 0, 0] Predicted [5, 7, 0, 0]
Input [0, 6, 4, 3] Expected: [0, 6, 0, 0] Predicted [0, 6, 0, 0]
